In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from k_fold import k_fold
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
# from feature_selection import filter_method as ft

In [19]:
import pandas as pd
df = pd.read_csv('./data/processed_mrna_zscore.csv')
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

In [40]:
df.iloc[:, -2]

0        47.033333
1        20.433333
2       138.133333
3       119.800000
4        43.833333
           ...    
1959     57.300000
1960    185.766667
1961    191.466667
1962    300.700000
1963    282.300000
Name: OS_MONTHS, Length: 1964, dtype: float64

In [77]:
threshold1 = 12
threshold2 = 36
labels = ['short', 'medium', 'long']
y = pd.cut(df['OS_MONTHS'], bins=[-float('inf'), threshold1, threshold2, float('inf')], labels=labels, right=False)
class_counts = y.value_counts()
class_counts

OS_MONTHS
long      1703
medium     216
short       45
Name: count, dtype: int64

In [85]:
from sklearn.feature_selection import SelectKBest, f_classif
k_best = SelectKBest(score_func=f_classif, k=100)
# Fit and transform the feature matrix
X_kbest = k_best.fit_transform(X, y)

# Print the indices of the selected features
selector = SelectKBest(f_classif, k=500)
selector.fit(X, y)
selected_feature_f_scores = X.columns[selector.get_support()].to_list()
selected_feature_f_scores

['ATP6V1C2',
 'MRPS5',
 'PDIA6',
 'CEBPB',
 'MASTL',
 'HINT3',
 'DNAJC9',
 'MRFAP1L1',
 'PURA',
 'TTYH1',
 'GAL',
 'NDFIP1',
 'ONECUT2',
 'ANKAR',
 'CDCA5',
 'AUNIP',
 'CA12',
 'FAM83D',
 'YTHDC2',
 'MELK',
 'RPP40',
 'B3GALNT2',
 'MIS18A',
 'PHGDH',
 'RABEP1',
 'DLGAP5',
 'SLC7A5',
 'GLYATL2',
 'SENP5',
 'EYA3',
 'RIOK3',
 'G6PC3',
 'NOP16',
 'RBM28',
 'ABCA3',
 'KRT81',
 'AHNAK',
 'MAP3K21',
 'VEZF1',
 'RRM2',
 'ANKHD1-EIF4EBP3',
 'REEP5',
 'PJA2',
 'SPDL1',
 'CASC1',
 'RAI2',
 'ADCY1',
 'SLC4A7',
 'UCK2',
 'NCOA7',
 'SUOX',
 'ECE2',
 'AFF3',
 'CDK6',
 'CENPW',
 'SNRPA1',
 'TCEAL3',
 'C5orf34',
 'JADE2',
 'FOXA1',
 'RUNDC1',
 'ADGRG6',
 'STK32B',
 'C15orf39',
 'FUT3',
 'SLC35C1',
 'DEGS2',
 'DONSON',
 'FOXM1',
 'CXXC5',
 'SAPCD2',
 'MAATS1',
 'KIF20A',
 'EGFR',
 'CDK1',
 'HEXIM2',
 'SPC25',
 'GTPBP4',
 'BTRC',
 'STC2',
 'LAD1',
 'FAM120AOS',
 'NDRG1',
 'DRC3',
 'ABAT',
 'TFF1',
 'C2CD5',
 'PDCD5',
 'BCAM',
 'MDP1',
 'TMEM144',
 'C5',
 'CXCL8',
 'TBC1D9',
 'RAD50',
 'TMEM101',
 'KCNG1

In [84]:
# Full 500 genes
scores = k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1), 
                X=X[selected_feature_f_scores], y=y)

Fold 1: 0.8426395939086294
Fold 2: 0.8324873096446701
Fold 3: 0.8578680203045685
Fold 4: 0.8578680203045685
Fold 5: 0.8469387755102041
Fold 6: 0.8571428571428571
Fold 7: 0.8469387755102041
Fold 8: 0.8571428571428571
Fold 9: 0.8724489795918368
Fold 10: 0.8214285714285714
Average Weighted Accuracy: 0.8492903760488968


**Forward**

In [118]:
from sklearn.feature_selection import SequentialFeatureSelector

def run_seq(X, y,
        clf=RandomForestClassifier(n_estimators=100, n_jobs=-1),
         n_feats = 10, direction='forward', scoring='accuracy', cv=5):
    sfs = SequentialFeatureSelector(clf, n_features_to_select=n_feats, direction=direction, scoring=scoring, cv=cv, n_jobs=-1)
    sfs.fit(X, y)
    selected_feature_names = X.columns[sfs.get_support()]
    return selected_feature_names


In [119]:
selected = run_seq(X[selected_feature_f_scores],y)

In [120]:
selected

Index(['BTRC', 'PARP3', 'USP6NL', 'PREX1', 'LARP6', 'FAM13B', 'TMEM170A',
       'CST3', 'FYCO1', 'ART3'],
      dtype='object')

In [122]:
# 10-gene
k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 
                X=X[['BTRC', 'PARP3', 'USP6NL', 'PREX1', 'LARP6', 'FAM13B', 'TMEM170A',
       'CST3', 'FYCO1', 'ART3']], y=y)

Fold 1: 0.7918781725888325
Fold 2: 0.7817258883248731
Fold 3: 0.7715736040609137
Fold 4: 0.7918781725888325
Fold 5: 0.8010204081632653
Fold 6: 0.7806122448979592
Fold 7: 0.7806122448979592
Fold 8: 0.8163265306122449
Fold 9: 0.8010204081632653
Fold 10: 0.8061224489795918
Average Weighted Accuracy: 0.7922770123277737


0.7922770123277737

Select 20 out of 500, take more than 1 hour

In [123]:
selected = run_seq(X[selected_feature_f_scores],y, n_feats=20)

In [124]:
selected

Index(['ONECUT2', 'SUOX', 'RUNDC1', 'MTPAP', 'SKP2', 'ALG3', 'E2F4', 'KRT6A',
       'CENPL', 'ARPC5L', 'SKA1', 'FSIP1', 'GAMT', 'CIAPIN1', 'PCBD2', 'CST3',
       'OIP5', 'PPM1J', 'NOSTRIN', 'EXOC6'],
      dtype='object')

In [125]:
# 20 genes
k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 
                X=X[['ONECUT2', 'SUOX', 'RUNDC1', 'MTPAP', 'SKP2', 'ALG3', 'E2F4', 'KRT6A',
       'CENPL', 'ARPC5L', 'SKA1', 'FSIP1', 'GAMT', 'CIAPIN1', 'PCBD2', 'CST3',
       'OIP5', 'PPM1J', 'NOSTRIN', 'EXOC6']], y=y)

Fold 1: 0.7969543147208121
Fold 2: 0.7868020304568528
Fold 3: 0.8274111675126904
Fold 4: 0.8324873096446701
Fold 5: 0.7908163265306123
Fold 6: 0.8163265306122449
Fold 7: 0.8214285714285714
Fold 8: 0.826530612244898
Fold 9: 0.8316326530612245
Fold 10: 0.7908163265306123
Average Weighted Accuracy: 0.8121205842743189


0.8121205842743189

**Backward (SKIP: Can not improve the accuracy)**

In [33]:
sfs2 = SFS(RandomForestClassifier(n_jobs=-1,n_estimators=5), 
           k_features=15, 
           forward=False, 
           floating=False, 
           verbose=1,
           scoring='accuracy',
           cv=3)

sfs2 = sfs2.fit(np.array(X[selected_feature_f_scores].fillna(0)), y)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.6s
Features: 99/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.7s
Features: 98/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.8s
Features: 97/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.8s
Features: 96/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.6s
Features: 95/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.3s
Features: 94/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.6s
Features: 93/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.5s
Features: 92/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.6s
Features: 91/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.8s
Features: 90/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.7s
Features: 89/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.4s
Features: 88/15[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.8s
Features: 87/15[Parallel(n_

In [34]:
selected_feat2= X[selected_feature_f_scores].columns[list(sfs2.k_feature_idx_)]
selected_feat2

Index(['ASIC5', 'COLEC10', 'TMEM29', 'KCNG1', 'ZNF501', 'OR7D2', 'ACTG1P20',
       'WAKMAR2', 'TM4SF4', 'KLHDC2', 'CDHR1', 'FGD3', 'OLFM4', 'PDCD2',
       'ESR1'],
      dtype='object')

In [35]:
scores = k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1), 
                X=X[selected_feat2], y=y)

Fold 1: 0.8274111675126904
Fold 2: 0.7817258883248731
Fold 3: 0.8121827411167513
Fold 4: 0.8071065989847716
Fold 5: 0.7857142857142857
Fold 6: 0.8163265306122449
Fold 7: 0.8316326530612245
Fold 8: 0.7704081632653061
Fold 9: 0.8163265306122449
Fold 10: 0.8469387755102041
Average Weighted Accuracy: 0.8095773334714597


**Exhaustive Feature Selection (SKIP)**

In [ ]:
efs1 = EFS(RandomForestClassifier(n_jobs=-1,n_estimators=5, random_state=0), 
           max_features=10,
           min_features=10,
           scoring='accuracy',
           print_progress=True,
           cv=2, n_jobs=-1)


efs1 = efs1.fit(np.array(X[selected_feature_f_scores].fillna(0)), y)

In [ ]:
selected_feat3= X[selected_feature_f_scores].columns[list(efs1.best_idx_)]
selected_feat3

**Recursive Feature Elimination**

In [129]:
from sklearn.feature_selection import RFE
rfe_selector = RFE(
    estimator=RandomForestClassifier(n_jobs=-1,n_estimators=100, random_state=0), 
    n_features_to_select=30)

rfe_selector.fit_transform(X[selected_feature_f_scores], y)
selected_feat4 = X[selected_feature_f_scores].columns[rfe_selector.support_].tolist()
selected_feat4

['RBM28',
 'SLC35C1',
 'GFRA1',
 'ZNF670',
 'B4GALT5',
 'PIGV',
 'RPL6',
 'SOX11',
 'ZNF750',
 'ACADSB',
 'PREX1',
 'RASAL1',
 'CENPL',
 'BCL2',
 'KNL1',
 'PTEN',
 'PPP1R14B',
 'ADGRG1',
 'AVP',
 'CALB2',
 'PGR',
 'CA9',
 'NUP155',
 'CSNK2A3',
 'HPDL',
 'ELOA',
 'NUDT19',
 'INAVA',
 'HMGA1',
 'C12orf45']

In [130]:
# The selected features from previous execution with different n_features_to_select

top15 = ['SLC35C1', 'ZNF670', 'B4GALT5','RPL6','ZNF750','PREX1','CENPL','PTEN','ADGRG1','AVP','CA9','ELOA','INAVA','HMGA1','C12orf45']
top30 = ['RBM28','SLC35C1','GFRA1','ZNF670','B4GALT5','PIGV','RPL6','SOX11','ZNF750','ACADSB','PREX1','RASAL1','CENPL','BCL2','KNL1','PTEN','PPP1R14B','ADGRG1','AVP','CALB2','PGR','CA9','NUP155','CSNK2A3','HPDL','ELOA','NUDT19','INAVA','HMGA1','C12orf45']


In [131]:
scores = k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 
                X=X[top30], y=y)

Fold 1: 0.8223350253807107
Fold 2: 0.7868020304568528
Fold 3: 0.8477157360406091
Fold 4: 0.8324873096446701
Fold 5: 0.8214285714285714
Fold 6: 0.8418367346938775
Fold 7: 0.8367346938775511
Fold 8: 0.8520408163265306
Fold 9: 0.8826530612244898
Fold 10: 0.8010204081632653
Average Weighted Accuracy: 0.8325054387237127


In [132]:
scores = k_fold(classifier=RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 
                X=X[top15], y=y)

Fold 1: 0.7868020304568528
Fold 2: 0.7969543147208121
Fold 3: 0.817258883248731
Fold 4: 0.8121827411167513
Fold 5: 0.8367346938775511
Fold 6: 0.8367346938775511
Fold 7: 0.8316326530612245
Fold 8: 0.8367346938775511
Fold 9: 0.8469387755102041
Fold 10: 0.7908163265306123
Average Weighted Accuracy: 0.8192789806277843
